In [80]:
import numpy as np
import pandas as pd
import glob
from matplotlib import pyplot as plt
import pickle

import sys,os
sys.path.insert(0, '..')
sys.path.insert(0, '../../util')

import helper as hp

In [153]:
dataset_to_labels = {
    'cifar10': ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
    
    'utkface': ['0-15', '15-25', '25-40', '40-60', '60+'],

    'utkface_race': ['0-15', '15-25', '25-40', '40-60', '60+'],

    'utkface_gender': ['0-15', '15-25', '25-40', '40-60', '60+'],
    
    'adience': ['0-2', '4-6', '8-13', '15-20', '25-32', '38-43', '48-53', '60-'],
    
    'cifar100': ['beaver', 'dolphin', 'otter', 'seal', 'whale',
        'aquarium_fish', 'flatfish', 'ray', 'shark', 'trout',
        'orchid', 'poppy', 'rose', 'sunflower', 'tulip',
        'bottle', 'bowl', 'can', 'cup', 'plate',
        'apple', 'mushroom', 'orange', 'pear', 'sweet_pepper',
        'clock', 'keyboard', 'lamp', 'telephone', 'television',
        'bed', 'chair', 'couch', 'table', 'wardrobe',
        'bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach',
        'bear', 'leopard', 'lion', 'tiger', 'wolf',
        'bridge', 'castle', 'house', 'road', 'skyscraper',
        'cloud', 'forest', 'mountain', 'plain', 'sea',
        'camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo',
        'fox', 'porcupine', 'possum', 'raccoon', 'skunk',
        'crab', 'lobster', 'snail', 'spider', 'worm',
        'baby', 'boy', 'girl', 'man', 'woman',
        'crocodile', 'dinosaur', 'lizard', 'snake', 'turtle',
        'hamster', 'mouse', 'rabbit', 'shrew', 'squirrel',
        'maple_tree', 'oak_tree', 'palm_tree', 'pine_tree', 'willow_tree',
        'bicycle', 'bus', 'motorcycle', 'pickup_truck', 'train',
        'lawn_mower', 'rocket', 'streetcar', 'tank', 'tractor'],
    
    'cifar100super': ['aquatic_mammals', 'fish', 'flowers', 'food_containers',
        'fruit_and_vegetables', 'household_electrical_devices',
        'household_furniture', 'insects', 'large_carnivores',
        'large_man-made_outdoor_things', 'large_natural_outdoor_scenes',
        'large_omnivores_and_herbivores', 'medium_mammals',
        'non-insect_invertebrates', 'people', 'reptiles', 'small_mammals',
        'trees', 'vehicles_1', 'vehicles_2']
}

dataset_to_root_dir = {
    'cifar10': '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/cifar10',
    'utkface': '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/utkface',
    'adience': '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/adience',
    'cifar100': '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/cifar100',
    'cifar100super': '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/cifar100super'
}

dataset_to_sens_attrs = {
    'cifar10': ['label'],
    'utkface': ['label', 'gender', 'race'],
    'adience': ['label', 'gender'],
    'cifar100': ['label'],
    'cifar100super': ['label']
}

dataset_to_model_names = {
    'utkface': ['utk_classifier', 'resnet', 'alexnet', 'vgg', 'densenet', 'squeezenet'],
    'cifar10': ['deep_cnn', 'pyramidnet', 'resnet', 'vgg', 'densenet', 'squeezenet'],
    'cifar100': ['deep_cnn_cifar100', 'pyramidnet', 'resnet', 'vgg', 'densenet', 'squeezenet'],
    'cifar100super': ['deep_cnn_cifar100', 'pyramidnet', 'resnet', 'vgg', 'densenet', 'squeezenet'],
    'adience': ['adience_classifier', 'resnet', 'alexnet', 'vgg', 'densenet', 'squeezenet']
}

sensitive_attrs = {'adience' : ['Female'],
                   'utkface_gender': ['Female'],
                   'utkface_race': ['white','black','asian','indian','other']
                  }


In [40]:
dataset = 'Adience'
labels = dataset_to_labels[dataset]
root_dir = dataset_to_root_dir[dataset]
sens_attrs = dataset_to_sens_attrs[dataset]
models = dataset_to_model_names[dataset]
# lb_files = glob.glob('{}/*'.format(root_dir))
lb_files = ['{}/{}_output'.format(root_dir, m) for m in models]
lb_files_epoch_0 = ['{}/{}_epoch_0_output'.format(root_dir, m) for m in models]

paper_friendly_plots = True

COLORS = ['blue', 'red', 'green', 'orange', 'magenta', 'yellow', 'black', 'grey', 'cyan']

In [41]:
ds_obj, datasets, data_loaders = \
    hp.get_data_loder_objects(dataset, ['test'], root_dir='..', **hp.get_loader_kwargs(batch_size=100))

In [42]:
lb_files, lb_files_epoch_0

(['/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/adience/adience_classifier_output',
  '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/adience/resnet_output',
  '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/adience/alexnet_output',
  '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/adience/vgg_output',
  '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/adience/densenet_output',
  '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/adience/squeezenet_output'],
 ['/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/adience/adience_classifier_epoch_0_output',
  '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/adience/resnet_epoch_0_output',
  '/NS/twitter-7/work/vnanda/adversarial_disparity/microsoft-smoothing/code/adience/alexnet_epoch_0_output',
  '/NS/twitter-7/work/vnanda/adversarial_dispa

In [43]:
def set_paper_friendly_plots_params():
    plt.style.use('seaborn-paper')
    plt.rcParams['font.size'] = 10
    plt.rcParams['axes.labelsize'] = 22
    plt.rcParams['axes.labelweight'] = 'bold'
    plt.rcParams['axes.titlesize'] = 15
    plt.rcParams['axes.linewidth'] = 1.25
    plt.rcParams['xtick.labelsize'] = 16
    plt.rcParams['ytick.labelsize'] = 16
    plt.rcParams['legend.fontsize'] = 20
    plt.rcParams['figure.titlesize'] = 22
    plt.rcParams['lines.linewidth'] = 4.0
    plt.rcParams['grid.color'] = 'grey'
    plt.rcParams['grid.linestyle'] = '--'
    plt.rcParams['grid.linewidth'] = 0.25
    plt.rcParams['figure.dpi'] = 50
    plt.rcParams['savefig.dpi'] = 50

In [44]:
heatmaps_for_sens_attrs, heatmaps_for_sens_attrs_epoch_0, x_labels, x_labels_epoch_0 = [], [], [], []

for sens_attr in sens_attrs:
    
    if sens_attr == 'gender':

        heatmaps_for_sens_attrs.append(np.zeros((len(models), len(ds_obj.genders))))
        heatmaps_for_sens_attrs_epoch_0.append(np.zeros((len(models), len(ds_obj.genders))))

        x_labels.append([ds_obj.get_image_protected_id_to_label(_id, 'gender') for _id in ds_obj.genders])
        x_labels_epoch_0.append([ds_obj.get_image_protected_id_to_label(_id, 'gender') for _id in ds_obj.genders])

    elif sens_attr == 'race':
        
        heatmaps_for_sens_attrs.append(np.zeros((len(models), len(ds_obj.races))))
        heatmaps_for_sens_attrs_epoch_0.append(np.zeros((len(models), len(ds_obj.races))))
        
        x_labels.append([ds_obj.get_image_protected_id_to_label(_id, 'race') for _id in ds_obj.races])
        x_labels_epoch_0.append([ds_obj.get_image_protected_id_to_label(_id, 'race') for _id in ds_obj.races])

    elif sens_attr == 'label':
        
        heatmaps_for_sens_attrs.append(np.zeros((len(models), len(ds_obj.classes))))
        heatmaps_for_sens_attrs_epoch_0.append(np.zeros((len(models), len(ds_obj.classes))))
        
        x_labels.append(ds_obj.classes)
        x_labels_epoch_0.append(ds_obj.classes)

    else:
        raise ValueError('Sens attr {} not supported!'.format(sens_attr))

In [45]:
x_labels, x_labels_epoch_0

([['0-2', '4-6', '8-13', '15-20', '25-32', '38-43', '48-53', '60-'],
  ['male', 'female']],
 [['0-2', '4-6', '8-13', '15-20', '25-32', '38-43', '48-53', '60-'],
  ['male', 'female']])

In [46]:
for model_idx, file in enumerate(lb_files):
    for heatmap_idx, sens_attr in enumerate(sens_attrs):
        df = pd.read_csv(file, sep='\t')
        print (len(df))
        try:
            df = df[df['label'] == df['base_prediction']] # take only correct predictions
        except:
            df = df[df['label'] == df['base']] # take only correct predictions
        print (len(df))
        unique, counts = np.unique(df['label'].to_numpy(), return_counts=True)
        print (dict(zip(unique, counts)))
        range_lb = np.linspace(np.min(df['radius']), np.max(df['radius']), 100)
        
        if paper_friendly_plots:
            set_paper_friendly_plots_params()
        
        for idx, attr in enumerate(np.unique(df[sens_attr].to_numpy())):
            minority_radii, majority_radii = df['radius'][df[sens_attr] == attr], \
                df['radius'][df[sens_attr] != attr]
            minority_cdf_vals = np.array([np.sum(minority_radii > t)/len(minority_radii) for t in range_lb])
            majority_cdf_vals = np.array([np.sum(majority_radii > t)/len(majority_radii) for t in range_lb])
            dt = range_lb[1] - range_lb[0]
            minority_auc = np.sum(minority_cdf_vals * dt)
            majority_auc = np.sum(majority_cdf_vals * dt)
            
            if sens_attr == 'label':
                y_coord = int(attr)
            else:
                y_coord = ds_obj.get_image_protected_label_to_id(attr, sens_attr)

            heatmaps_for_sens_attrs[heatmap_idx][model_idx, y_coord] = (minority_auc - majority_auc)/majority_auc

for model_idx, file in enumerate(lb_files_epoch_0):
    for heatmap_idx, sens_attr in enumerate(sens_attrs):
        df = pd.read_csv(file, sep='\t')
        print (len(df))
        try:
            df = df[df['label'] == df['base_prediction']] # take only correct predictions
        except:
            df = df[df['label'] == df['base']] # take only correct predictions
        print (len(df))
        unique, counts = np.unique(df['label'].to_numpy(), return_counts=True)
        print (file.split('/')[-1], dict(zip(unique, counts)))
        range_lb = np.linspace(np.min(df['radius']), np.max(df['radius']), 100)
        
        if paper_friendly_plots:
            set_paper_friendly_plots_params()
        
        for idx, attr in enumerate(np.unique(df[sens_attr].to_numpy())):
            minority_radii, majority_radii = df['radius'][df[sens_attr] == attr], \
                df['radius'][df[sens_attr] != attr]
            minority_cdf_vals = np.array([np.sum(minority_radii > t)/len(minority_radii) for t in range_lb])
            majority_cdf_vals = np.array([np.sum(majority_radii > t)/len(majority_radii) for t in range_lb])
            dt = range_lb[1] - range_lb[0]
            minority_auc = np.sum(minority_cdf_vals * dt)
            majority_auc = np.sum(majority_cdf_vals * dt)
            
            if sens_attr == 'label':
                y_coord = int(attr)
            else:
                y_coord = ds_obj.get_image_protected_label_to_id(attr, sens_attr)
            
            sigma = (minority_auc - majority_auc)/majority_auc
            heatmaps_for_sens_attrs_epoch_0[heatmap_idx][model_idx, y_coord] = sigma \
                if not np.isnan(sigma) else 0.

689
324
{0: 12, 1: 80, 2: 18, 3: 5, 4: 144, 5: 48, 7: 17}
689
324
{0: 12, 1: 80, 2: 18, 3: 5, 4: 144, 5: 48, 7: 17}
689
359
{0: 27, 1: 66, 2: 42, 3: 13, 4: 131, 5: 40, 6: 8, 7: 32}
689
359
{0: 27, 1: 66, 2: 42, 3: 13, 4: 131, 5: 40, 6: 8, 7: 32}
689
318
{0: 14, 1: 75, 2: 10, 3: 4, 4: 150, 5: 35, 6: 2, 7: 28}
689
318
{0: 14, 1: 75, 2: 10, 3: 4, 4: 150, 5: 35, 6: 2, 7: 28}
689
367
{0: 20, 1: 63, 2: 44, 3: 13, 4: 155, 5: 39, 6: 8, 7: 25}
689
367
{0: 20, 1: 63, 2: 44, 3: 13, 4: 155, 5: 39, 6: 8, 7: 25}
689
360
{0: 29, 1: 57, 2: 32, 3: 11, 4: 159, 5: 32, 6: 11, 7: 29}
689
360
{0: 29, 1: 57, 2: 32, 3: 11, 4: 159, 5: 32, 6: 11, 7: 29}
689
341
{0: 25, 1: 73, 2: 12, 3: 10, 4: 174, 5: 8, 6: 11, 7: 28}
689
341
{0: 25, 1: 73, 2: 12, 3: 10, 4: 174, 5: 8, 6: 11, 7: 28}
594
215
adience_classifier_epoch_0_output {0: 7, 1: 58, 2: 11, 4: 139}
594
215
adience_classifier_epoch_0_output {0: 7, 1: 58, 2: 11, 4: 139}
689
343
resnet_epoch_0_output {0: 21, 1: 74, 2: 25, 3: 3, 4: 176, 5: 33, 7: 11}
689
343
resn

In [47]:
heatmaps_for_sens_attrs[0].shape[1]

8

In [48]:
heatmaps_for_sens_attrs

[array([[-0.04418723,  0.06131513, -0.20665834, -0.26822911,  0.04665226,
          0.00365964,  0.        , -0.12663621],
        [ 0.91656655, -0.3073543 , -0.1015804 , -0.09825005, -0.01746787,
         -0.07239426, -0.43260578,  0.30514157],
        [ 0.01387718,  0.0061822 , -0.20503438, -0.15728881,  0.10419667,
         -0.15190307, -0.01009803, -0.03733545],
        [ 0.10031361, -0.1099014 , -0.3382359 , -0.16910042,  0.29956937,
         -0.22453583, -0.39028533,  0.30817232],
        [ 0.34475181, -0.19571394, -0.08602015, -0.30644624,  0.18485832,
         -0.15518806, -0.0418889 , -0.13228473],
        [ 0.24438214, -0.31670719, -0.26033333, -0.24386946,  0.4949874 ,
         -0.50257348, -0.64404008, -0.12874147]]),
 array([[ 0.02479698, -0.02419696],
        [-0.1888631 ,  0.23283751],
        [-0.00686181,  0.00690922],
        [-0.01534866,  0.01558791],
        [ 0.02485732, -0.02425442],
        [-0.16327495,  0.19513573]])]

## Plotting the heatmaps

In [49]:
from matplotlib.gridspec import GridSpec
from matplotlib import colors

In [107]:
def func(dataset):
    labels = dataset_to_labels[dataset]
    root_dir = dataset_to_root_dir[dataset]
    sens_attrs = dataset_to_sens_attrs[dataset]
    models = dataset_to_model_names[dataset]
    # lb_files = glob.glob('{}/*'.format(root_dir))
    lb_files = ['{}/{}_output'.format(root_dir, m) for m in models]
    lb_files_epoch_0 = ['{}/{}_epoch_0_output'.format(root_dir, m) for m in models]

    paper_friendly_plots = True

    COLORS = ['blue', 'red', 'green', 'orange', 'magenta', 'yellow', 'black', 'grey', 'cyan']

    ds_obj, datasets, data_loaders = \
        hp.get_data_loder_objects(dataset, ['test'], root_dir='..', **hp.get_loader_kwargs(batch_size=100))  

    heatmaps_for_sens_attrs, heatmaps_for_sens_attrs_epoch_0, x_labels, x_labels_epoch_0 = [], [], [], []

    for sens_attr in sens_attrs:

        if sens_attr == 'gender':

            heatmaps_for_sens_attrs.append(np.zeros((len(models), len(ds_obj.genders))))
            heatmaps_for_sens_attrs_epoch_0.append(np.zeros((len(models), len(ds_obj.genders))))

            x_labels.append([ds_obj.get_image_protected_id_to_label(_id, 'gender') for _id in ds_obj.genders])
            x_labels_epoch_0.append([ds_obj.get_image_protected_id_to_label(_id, 'gender') for _id in ds_obj.genders])

        elif sens_attr == 'race':

            heatmaps_for_sens_attrs.append(np.zeros((len(models), len(ds_obj.races))))
            heatmaps_for_sens_attrs_epoch_0.append(np.zeros((len(models), len(ds_obj.races))))

            x_labels.append([ds_obj.get_image_protected_id_to_label(_id, 'race') for _id in ds_obj.races])
            x_labels_epoch_0.append([ds_obj.get_image_protected_id_to_label(_id, 'race') for _id in ds_obj.races])

        elif sens_attr == 'label':

            heatmaps_for_sens_attrs.append(np.zeros((len(models), len(ds_obj.classes))))
            heatmaps_for_sens_attrs_epoch_0.append(np.zeros((len(models), len(ds_obj.classes))))

            x_labels.append(ds_obj.classes)
            x_labels_epoch_0.append(ds_obj.classes)

        else:
            raise ValueError('Sens attr {} not supported!'.format(sens_attr))


    for model_idx, file in enumerate(lb_files):
        for heatmap_idx, sens_attr in enumerate(sens_attrs):
            df = pd.read_csv(file, sep='\t')
            print (len(df))
            try:
                df = df[df['label'] == df['base_prediction']] # take only correct predictions
            except:
                df = df[df['label'] == df['base']] # take only correct predictions
            print (len(df))
            unique, counts = np.unique(df['label'].to_numpy(), return_counts=True)
            print (dict(zip(unique, counts)))
            range_lb = np.linspace(np.min(df['radius']), np.max(df['radius']), 100)

            if paper_friendly_plots:
                set_paper_friendly_plots_params()

            for idx, attr in enumerate(np.unique(df[sens_attr].to_numpy())):
                minority_radii, majority_radii = df['radius'][df[sens_attr] == attr], \
                    df['radius'][df[sens_attr] != attr]
                minority_cdf_vals = np.array([np.sum(minority_radii > t)/len(minority_radii) for t in range_lb])
                majority_cdf_vals = np.array([np.sum(majority_radii > t)/len(majority_radii) for t in range_lb])
                dt = range_lb[1] - range_lb[0]
                minority_auc = np.sum(minority_cdf_vals * dt)
                majority_auc = np.sum(majority_cdf_vals * dt)

                if sens_attr == 'label':
                    y_coord = int(attr)
                else:
                    y_coord = ds_obj.get_image_protected_label_to_id(attr, sens_attr)

                heatmaps_for_sens_attrs[heatmap_idx][model_idx, y_coord] = (minority_auc - majority_auc)/majority_auc

    for model_idx, file in enumerate(lb_files_epoch_0):
        for heatmap_idx, sens_attr in enumerate(sens_attrs):
            df = pd.read_csv(file, sep='\t')
            print (len(df))
            try:
                df = df[df['label'] == df['base_prediction']] # take only correct predictions
            except:
                df = df[df['label'] == df['base']] # take only correct predictions
            print (len(df))
            unique, counts = np.unique(df['label'].to_numpy(), return_counts=True)
            print (file.split('/')[-1], dict(zip(unique, counts)))
            range_lb = np.linspace(np.min(df['radius']), np.max(df['radius']), 100)

            if paper_friendly_plots:
                set_paper_friendly_plots_params()

            for idx, attr in enumerate(np.unique(df[sens_attr].to_numpy())):
                minority_radii, majority_radii = df['radius'][df[sens_attr] == attr], \
                    df['radius'][df[sens_attr] != attr]
                minority_cdf_vals = np.array([np.sum(minority_radii > t)/len(minority_radii) for t in range_lb])
                majority_cdf_vals = np.array([np.sum(majority_radii > t)/len(majority_radii) for t in range_lb])
                dt = range_lb[1] - range_lb[0]
                minority_auc = np.sum(minority_cdf_vals * dt)
                majority_auc = np.sum(majority_cdf_vals * dt)

                if sens_attr == 'label':
                    y_coord = int(attr)
                else:
                    y_coord = ds_obj.get_image_protected_label_to_id(attr, sens_attr)

                sigma = (minority_auc - majority_auc)/majority_auc
                heatmaps_for_sens_attrs_epoch_0[heatmap_idx][model_idx, y_coord] = sigma \
                    if not np.isnan(sigma) else 0.
            
    return heatmaps_for_sens_attrs




In [149]:
def plot_ds(ds_name, models, classes, attack):

    a = len(classes[ds_name.lower()])
    b = len(models)

    best_arr = np.zeros((a,b))
    frst_arr = np.zeros((a,b))
    
    for i,model in enumerate(models):
        ### find the middle epoch of training 
        trained_models = glob.glob('../../{}/model_weights/{}'.format(ds_name, 
                                                           '{}_epoch_*[0-9]_lr_*.pth'.format(model)))
        directory = '../plots/{}/{}/{}'.format(ds_name,model,attack)

        for j,sens_attr in enumerate(classes[ds_name.lower()]):


            best_cdf, mid_cdf, fst_cdf = None, None, None

            best_fn_w_epoch  = '{}/cdfs_{}_epoch_best.dat'.format(directory,sens_attr)
            best_fn_wo_epoch = '{}/cdfs_{}.dat'.format(directory,sens_attr)
            if os.path.exists(best_fn_w_epoch):
                with open(best_fn_w_epoch,'rb') as f:
                    best_cdf = pickle.load(f)
            elif os.path.exists(best_fn_wo_epoch):
                with open(best_fn_wo_epoch,'rb') as f:
                    best_cdf = pickle.load(f)

            if best_cdf:
                dt = (best_cdf['taus'][1]-best_cdf['taus'][0])
                minority = sum(best_cdf['minority']*dt)
                majority = sum(best_cdf['majority']*dt)

                best_arr[j,i] = (minority-majority)/majority


            first_fn_w_epoch = '{}/cdfs_{}_epoch_{}.dat'.format(directory,sens_attr,'0')
            if os.path.exists(first_fn_w_epoch):
                with open(first_fn_w_epoch,'rb') as f:
                    fst_cdf = pickle.load(f)

            if fst_cdf:
                dt = (fst_cdf['taus'][1]-fst_cdf['taus'][0])
                minority = sum(fst_cdf['minority']*dt)
                majority = sum(fst_cdf['majority']*dt)

                frst_arr[j,i] = (minority-majority)/majority
                
    return best_arr, frst_arr

df = 'DeepFool'
cw = 'CarliniWagner'

In [138]:
arrs_cifar10 = func('cifar10')
arrs_cifar100 = func('cifar100')
arrs_cifar100super = func('cifar100super')
arrs_adience = func('adience')
arrs_utkface = func('utkface')

2000
1726
{0: 158, 1: 192, 2: 166, 3: 168, 4: 186, 5: 140, 6: 183, 7: 170, 8: 182, 9: 181}
2000
1719
{0: 170, 1: 195, 2: 168, 3: 150, 4: 174, 5: 143, 6: 171, 7: 181, 8: 184, 9: 183}
2000
1665
{0: 160, 1: 185, 2: 173, 3: 157, 4: 172, 5: 133, 6: 167, 7: 177, 8: 174, 9: 167}
2000
1788
{0: 164, 1: 202, 2: 186, 3: 165, 4: 182, 5: 146, 6: 183, 7: 185, 8: 186, 9: 189}
2000
1673
{0: 158, 1: 191, 2: 154, 3: 142, 4: 169, 5: 146, 6: 182, 7: 178, 8: 179, 9: 174}
2000
1539
{0: 151, 1: 166, 2: 133, 3: 130, 4: 165, 5: 108, 6: 169, 7: 159, 8: 177, 9: 181}
2000
197
deep_cnn_epoch_0_output {4: 197}
160
89
pyramidnet_epoch_0_output {0: 3, 1: 10, 2: 5, 3: 5, 4: 5, 5: 9, 6: 13, 7: 11, 8: 10, 9: 18}


/NS/twitter-7/work/vnanda/miniconda3/envs/adv_disparity/lib/python3.7/site-packages/ipykernel_launcher.py:101: RuntimeWarning: invalid value encountered in long_scalars


2000
1480
resnet_epoch_0_output {0: 143, 1: 167, 2: 121, 3: 142, 4: 159, 5: 99, 6: 156, 7: 158, 8: 173, 9: 162}
877
702
vgg_epoch_0_output {0: 65, 1: 80, 2: 62, 3: 45, 4: 87, 5: 53, 6: 78, 7: 77, 8: 77, 9: 78}
2000
1558
densenet_epoch_0_output {0: 158, 1: 188, 2: 139, 3: 114, 4: 167, 5: 112, 6: 165, 7: 171, 8: 175, 9: 169}
2000
205
squeezenet_epoch_0_output {7: 205}


/NS/twitter-7/work/vnanda/miniconda3/envs/adv_disparity/lib/python3.7/site-packages/ipykernel_launcher.py:101: RuntimeWarning: invalid value encountered in long_scalars


Files already downloaded and verified
Files already downloaded and verified
2000
1193
{0: 23, 1: 19, 2: 9, 3: 6, 4: 10, 5: 5, 6: 9, 7: 6, 8: 10, 9: 20, 10: 8, 11: 5, 12: 21, 13: 5, 14: 7, 15: 11, 16: 22, 17: 12, 18: 10, 19: 7, 20: 14, 21: 16, 22: 10, 23: 14, 24: 16, 25: 9, 26: 10, 27: 6, 28: 15, 29: 10, 30: 14, 31: 10, 32: 11, 33: 7, 34: 10, 35: 15, 36: 12, 37: 11, 38: 8, 39: 15, 40: 10, 41: 17, 42: 12, 43: 17, 44: 6, 45: 9, 46: 8, 47: 20, 48: 18, 49: 18, 50: 7, 51: 7, 52: 13, 53: 19, 54: 15, 55: 8, 56: 14, 57: 17, 58: 14, 59: 7, 60: 20, 61: 17, 62: 8, 63: 11, 64: 7, 65: 8, 66: 18, 67: 10, 68: 16, 69: 16, 70: 22, 71: 16, 72: 4, 73: 7, 74: 7, 75: 12, 76: 15, 77: 12, 78: 8, 79: 18, 80: 7, 81: 9, 82: 24, 83: 9, 84: 9, 85: 8, 86: 11, 87: 10, 88: 12, 89: 10, 90: 11, 91: 17, 92: 11, 93: 6, 94: 16, 95: 10, 96: 12, 97: 11, 98: 8, 99: 16}
2000
1133
{0: 23, 1: 16, 2: 9, 3: 6, 4: 10, 5: 8, 6: 8, 7: 10, 8: 10, 9: 16, 10: 8, 11: 8, 12: 20, 13: 9, 14: 9, 15: 11, 16: 22, 17: 9, 18: 4, 19: 7, 20: 12, 

689
359
{0: 27, 1: 66, 2: 42, 3: 13, 4: 131, 5: 40, 6: 8, 7: 32}
689
318
{0: 14, 1: 75, 2: 10, 3: 4, 4: 150, 5: 35, 6: 2, 7: 28}
689
318
{0: 14, 1: 75, 2: 10, 3: 4, 4: 150, 5: 35, 6: 2, 7: 28}
689
367
{0: 20, 1: 63, 2: 44, 3: 13, 4: 155, 5: 39, 6: 8, 7: 25}
689
367
{0: 20, 1: 63, 2: 44, 3: 13, 4: 155, 5: 39, 6: 8, 7: 25}
689
360
{0: 29, 1: 57, 2: 32, 3: 11, 4: 159, 5: 32, 6: 11, 7: 29}
689
360
{0: 29, 1: 57, 2: 32, 3: 11, 4: 159, 5: 32, 6: 11, 7: 29}
689
341
{0: 25, 1: 73, 2: 12, 3: 10, 4: 174, 5: 8, 6: 11, 7: 28}
689
341
{0: 25, 1: 73, 2: 12, 3: 10, 4: 174, 5: 8, 6: 11, 7: 28}
613
229
adience_classifier_epoch_0_output {0: 7, 1: 58, 2: 11, 4: 153}
613
229
adience_classifier_epoch_0_output {0: 7, 1: 58, 2: 11, 4: 153}
689
343
resnet_epoch_0_output {0: 21, 1: 74, 2: 25, 3: 3, 4: 176, 5: 33, 7: 11}
689
343
resnet_epoch_0_output {0: 21, 1: 74, 2: 25, 3: 3, 4: 176, 5: 33, 7: 11}
689
318
alexnet_epoch_0_output {0: 14, 1: 75, 2: 10, 3: 4, 4: 150, 5: 35, 6: 2, 7: 28}
689
318
alexnet_epoch_0_ou

In [217]:
agg_df, total_df = 0,0
agg_cw, total_cw = 0,0

models = ['adience_classifier','resnet','alexnet','vgg','densenet','squeezenet']
best_arr_1, frst_arr_1 = plot_ds('Adience',models, dataset_to_labels, df)
best_arr_2, frst_arr_2 = plot_ds('Adience',models, sensitive_attrs, df)

best_arr_1_cw, frst_arr_1_cw = plot_ds('Adience',models, dataset_to_labels, cw)
best_arr_2_cw, frst_arr_2_cw = plot_ds('Adience',models, sensitive_attrs, cw)

agg_df += np.sum(np.equal(best_arr_1.T > 0, arrs_adience[0]>0))
total_df += np.prod(best_arr_1.shape)
agg_df += np.sum(np.equal(best_arr_2.T > 0, arrs_adience[1]>0))
total_df += np.prod(best_arr_2.shape)

agg_cw += np.sum(np.equal(best_arr_1_cw.T > 0, arrs_adience[0]>0))
total_cw += np.prod(best_arr_1.shape)
agg_cw += np.sum(np.equal(best_arr_2_cw.T > 0, arrs_adience[1]>0))
total_cw += np.prod(best_arr_2.shape)

print(agg_df,total_df,agg_cw,total_cw)
print(agg_df/total_df*100,agg_cw/total_cw*100)


agg_df, total_df = 0,0
agg_cw, total_cw = 0,0

models = ['utk_classifier','resnet','alexnet','vgg','densenet','squeezenet']
best_arr_1, frst_arr_1 = plot_ds('utkface_race',models, dataset_to_labels, df)
best_arr_2, frst_arr_2 = plot_ds('utkface_gender',models, sensitive_attrs, df)
best_arr_3, frst_arr_3 = plot_ds('utkface_race',models, sensitive_attrs, df)

best_arr_1_cw, frst_arr_1_cw = plot_ds('utkface_race',models, dataset_to_labels, cw)
best_arr_2_cw, frst_arr_2_cw = plot_ds('utkface_gender',models, sensitive_attrs, cw)
best_arr_3_cw, frst_arr_3_cw = plot_ds('utkface_race',models, sensitive_attrs, cw)

agg_df += np.sum(np.equal(best_arr_1.T > 0, arrs_utkface[0]>0))
total_df += np.prod(best_arr_1.shape)
agg_df += np.sum(np.equal(best_arr_2[0] > 0, arrs_utkface[1][:,1]>0))
total_df += np.prod(best_arr_2.shape)
agg_df += np.sum(np.equal(best_arr_3.T > 0, arrs_utkface[2]>0))
total_df += np.prod(best_arr_3.shape)

agg_cw += np.sum(np.equal(best_arr_1_cw.T > 0, arrs_utkface[0]>0))
total_cw += np.prod(best_arr_1_cw.shape)
agg_cw += np.sum(np.equal(best_arr_2_cw[0] > 0, arrs_utkface[1][:,1]>0))
total_cw += np.prod(best_arr_2_cw.shape)
agg_cw += np.sum(np.equal(best_arr_3_cw.T > 0, arrs_utkface[2]>0))
total_cw += np.prod(best_arr_3_cw.shape)

print(agg_df,total_df,agg_cw,total_cw)
print(agg_df/total_df*100,agg_cw/total_cw*100)


agg_df, total_df = 0,0
agg_cw, total_cw = 0,0

models = ['deep_cnn','pyramidnet','resnet','vgg','densenet','squeezenet']
models_full = ['deep_cnn','pyramidnet_alpha_64_depth_110','resnet','vgg','densenet','squeezenet']
best_arr_1, frst_arr_1 = plot_ds('CIFAR10',models_full, dataset_to_labels, df)

best_arr_1_cw, frst_arr_1_cw = plot_ds('CIFAR10',models_full, dataset_to_labels, cw)

agg_df += np.sum(np.equal(best_arr_1.T > 0, arrs_cifar10[0]>0))
total_df += np.prod(best_arr_1.shape)
agg_cw += np.sum(np.equal(best_arr_1_cw.T > 0, arrs_cifar10[0]>0))
total_cw += np.prod(best_arr_1.shape)

print(agg_df,total_df,agg_cw,total_cw)
print(agg_df/total_df*100,agg_cw/total_cw*100)


agg_df, total_df = 0,0
agg_cw, total_cw = 0,0

models = ['deep_cnn','pyramidnet','resnet','vgg','densenet','squeezenet']
models_full = ['deep_cnn_cifar100','pyramidnet_alpha_48_depth_164_bottleneck','resnet','vgg','densenet','squeezenet']
best_arr_1, frst_arr_1 = plot_ds('CIFAR100',models_full, dataset_to_labels, df)

best_arr_1_cw, frst_arr_1_cw = plot_ds('CIFAR100',models_full, dataset_to_labels, cw)

agg_df += np.sum(np.equal(best_arr_1.T > 0, arrs_cifar100[0]>0))
total_df += np.prod(best_arr_1.shape)
agg_cw += np.sum(np.equal(best_arr_1_cw.T > 0, arrs_cifar100[0]>0))
total_cw += np.prod(best_arr_1.shape)

print(agg_df,total_df,agg_cw,total_cw)
print(agg_df/total_df*100,agg_cw/total_cw*100)


agg_df, total_df = 0,0
agg_cw, total_cw = 0,0

models = ['deep_cnn','pyramidnet','resnet','vgg','densenet','squeezenet']
models_full = ['deep_cnn_cifar100','pyramidnet_alpha_48_depth_164_bottleneck','resnet','vgg','densenet','squeezenet']
best_arr_1, frst_arr_1 = plot_ds('CIFAR100super',models_full, dataset_to_labels, df)

best_arr_1_cw, frst_arr_1_cw = plot_ds('CIFAR100super',models_full, dataset_to_labels, cw)

agg_df += np.sum(np.equal(best_arr_1.T > 0, arrs_cifar100super[0]>0))
total_df += np.prod(best_arr_1.shape)
agg_cw += np.sum(np.equal(best_arr_1_cw.T > 0, arrs_cifar100super[0]>0))
total_cw += np.prod(best_arr_1.shape)

print(agg_df,total_df,agg_cw,total_cw)
print(agg_df/total_df*100,agg_cw/total_cw*100)

/NS/twitter-7/work/vnanda/miniconda3/envs/adv_disparity/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater
  # This is added back by InteractiveShellApp.init_path()
/NS/twitter-7/work/vnanda/miniconda3/envs/adv_disparity/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in greater
  app.launch_new_instance()


45 54 43 54
83.33333333333334 79.62962962962963
49 66 48 66
74.24242424242425 72.72727272727273
45 60 46 60
75.0 76.66666666666667


/NS/twitter-7/work/vnanda/miniconda3/envs/adv_disparity/lib/python3.7/site-packages/ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in greater
/NS/twitter-7/work/vnanda/miniconda3/envs/adv_disparity/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in greater


323 600 324 600
53.833333333333336 54.0
79 120 80 120
65.83333333333333 66.66666666666666
